<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 2: Analyzing Chipotle Data

_Author: Joseph Nelson (DC)_

---

For Project 2, you will complete a series of exercises exploring [order data from Chipotle](https://github.com/TheUpshot/chipotle), compliments of _The New York Times'_ "The Upshot."

For these exercises, you will conduct basic exploratory data analysis (Pandas not required) to understand the essentials of Chipotle's order data: how many orders are being made, the average price per order, how many different ingredients are used, etc. These allow you to practice business analysis skills while also becoming comfortable with Python.

---

## Basic Level

### Part 1: Read in the file with `csv.reader()` and store it in an object called `file_nested_list`.

Hint: This is a TSV (tab-separated value) file, and `csv.reader()` needs to be told [how to handle it](https://docs.python.org/2/library/csv.html).

In [2]:
import csv
from collections import namedtuple   # Convenient to store the data rows

DATA_FILE = './data/chipotle.tsv'

In [3]:
file_nested_list = []
with open(DATA_FILE, 'r') as csvfile:
    loaded_data = csv.reader(csvfile, delimiter='\t')
    for i in loaded_data:
        file_nested_list.append(i)

### Part 2: Separate `file_nested_list` into the `header` and the `data`.


In [4]:
header = file_nested_list[0]
header

['order_id', 'quantity', 'item_name', 'choice_description', 'item_price']

In [18]:
data = file_nested_list[1:]
data[:8]

[['1', '1', 'Chips and Fresh Tomato Salsa', 'NULL', '$2.39 '],
 ['1', '1', 'Izze', '[Clementine]', '$3.39 '],
 ['1', '1', 'Nantucket Nectar', '[Apple]', '$3.39 '],
 ['1', '1', 'Chips and Tomatillo-Green Chili Salsa', 'NULL', '$2.39 '],
 ['2',
  '2',
  'Chicken Bowl',
  '[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]',
  '$16.98 '],
 ['3',
  '1',
  'Chicken Bowl',
  '[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sour Cream, Guacamole, Lettuce]]',
  '$10.98 '],
 ['3', '1', 'Side of Chips', 'NULL', '$1.69 '],
 ['4',
  '1',
  'Steak Burrito',
  '[Tomatillo Red Chili Salsa, [Fajita Vegetables, Black Beans, Pinto Beans, Cheese, Sour Cream, Guacamole, Lettuce]]',
  '$11.75 ']]

In [6]:
# Order = namedtuple('Order', 'order_id quantity item_name choice_description item_price')

---

## Intermediate Level

### Part 3: Calculate the average price of an order.

Hint: Examine the data to see if the `quantity` column is relevant to this calculation.

Hint: Think carefully about the simplest way to do this!

In [73]:
def avg_orders( arr ):
    current_item = 0
    holder = []
    final = []
    for order in arr:
        item_value = int(order[1]) * float( order[4].replace('$','') ) # value of current row
        if order[0] != current_item: # if moving onto new order...
            if int(current_item) > 0: # skip 0 value and
                final.append( sum(holder) / len(holder) ) # append average of current order
            holder = [] # reset current order list
            current_item = order[0] # change value of current order
            holder.append( item_value ) # append value of current row to order list
        else:
            holder.append( item_value ) # append value of current row to order list
    print( round(sum(final)/len(final), 2) ) # print average of all order's averages

In [74]:
avg_orders( data )

9.11


### Part 4: Create a list (or set) named `unique_sodas` containing all of unique sodas and soft drinks that Chipotle sells.

Note: Just look for `'Canned Soda'` and `'Canned Soft Drink'`, and ignore other drinks like `'Izze'`.

In [9]:
data[18][2]

'Canned Soda'

In [58]:
data[18][2] == 'Canned Soda'

True

In [10]:
str(data[18][2]) is 'Canned Soda'

False

In [65]:
unique_sodas =[]
for i in data:
    if i[2] == 'Canned Soda' or i[2] == 'Canned Soft Drink':
        soda = str(i[3]).replace('[','').replace(']','')
        if soda not in unique_sodas:
            unique_sodas.append(soda)
unique_sodas

['Sprite',
 'Dr. Pepper',
 'Mountain Dew',
 'Diet Dr. Pepper',
 'Coca Cola',
 'Diet Coke',
 'Coke',
 'Lemonade',
 'Nestea']

---

## Advanced Level


### Part 5: Calculate the average number of toppings per burrito.

Note: Let's ignore the `quantity` column to simplify this task.

Hint: Think carefully about the easiest way to count the number of toppings!


In [60]:
count = []
for item in data:
    if 'Burrito' in item[2]:
        toppings = item[3].replace('[','').replace(']','')
        toppings = toppings.split(',')
        count.append( len( toppings ))
print( sum(count) / len(count) )

5.395051194539249


### Part 6: Create a dictionary. Let the keys represent chip orders and the values represent the total number of orders.

Expected output: `{'Chips and Roasted Chili-Corn Salsa': 18, ... }`

Note: Please take the `quantity` column into account!

Optional: Learn how to use `.defaultdict()` to simplify your code.

In [61]:
from collections import defaultdict

In [63]:
chip_count = defaultdict(int)
for item in data:
    if 'Chips' in item[2]:
        chip_count[ item[2] ] += ( 1 * int(item[1]) )
chip_count

defaultdict(int,
            {'Chips': 230,
             'Chips and Fresh Tomato Salsa': 130,
             'Chips and Guacamole': 506,
             'Chips and Mild Fresh Tomato Salsa': 1,
             'Chips and Roasted Chili Corn Salsa': 23,
             'Chips and Roasted Chili-Corn Salsa': 18,
             'Chips and Tomatillo Green Chili Salsa': 45,
             'Chips and Tomatillo Red Chili Salsa': 50,
             'Chips and Tomatillo-Green Chili Salsa': 33,
             'Chips and Tomatillo-Red Chili Salsa': 25,
             'Side of Chips': 110})

---

## Bonus: Craft a problem statement about this data that interests you, and then answer it!
